### Sparse Regression analysis of Non linear dynamics of granular material

In [ ]:
import os
import torch
import glob
import numpy as np
import pandas as pd
from pathlib import Path
from torch import nn
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# File reading and processing: 
#-------------------------------------------------- Read LAMMPS Dump Files ------------------------------------#
def read_lammps_dump(filename, columns_range=('id', 'fz')):
    """
    Read LAMMPS dump file and extract data from specified column range.
    
    Parameters:
    -----------
    filename : str
        Path to the LAMMPS dump file
    columns_range : tuple, optional
        Tuple of (start_column, end_column) names to extract
        Default is ('id', 'fz')
    
    Returns:
    --------
    dict : Dictionary containing:
        - 'timesteps': list of timestep values
        - 'natoms': list of number of atoms per timestep
        - 'box_bounds': list of box boundary arrays per timestep
        - 'data': list of DataFrames with selected columns per timestep
        - 'all_data': Combined DataFrame with all timesteps (includes 'timestep' column)
    """
    
    timesteps = []
    natoms_list = []
    box_bounds_list = []
    data_list = []
    
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        # Check for timestep
        if line == "ITEM: TIMESTEP":
            timestep = int(lines[i+1].strip())
            timesteps.append(timestep)
            i += 2
            continue
        
        # Check for number of atoms
        if line == "ITEM: NUMBER OF ATOMS":
            natoms = int(lines[i+1].strip())
            natoms_list.append(natoms)
            i += 2
            continue
        
        # Check for box bounds
        if line.startswith("ITEM: BOX BOUNDS"):
            box_bounds = []
            for j in range(3):
                bounds = [float(x) for x in lines[i+1+j].strip().split()]
                box_bounds.append(bounds)
            box_bounds_list.append(np.array(box_bounds))
            i += 4
            continue
        
        # Check for atoms data
        if line.startswith("ITEM: ATOMS"):
            # Extract column names from header
            header_parts = line.split()
            column_names = header_parts[2:]  # Skip "ITEM:" and "ATOMS"
            
            # Find indices of start and end columns
            start_col = columns_range[0]
            end_col = columns_range[1]
            
            if start_col not in column_names:
                raise ValueError(f"Start column '{start_col}' not found in dump file")
            if end_col not in column_names:
                raise ValueError(f"End column '{end_col}' not found in dump file")
            
            start_idx = column_names.index(start_col)
            end_idx = column_names.index(end_col) + 1
            selected_columns = column_names[start_idx:end_idx]
            
            # Read atom data
            atom_data = []
            natoms_current = natoms_list[-1]
            for j in range(natoms_current):
                atom_line = lines[i+1+j].strip().split()
                # Extract only the selected column range
                selected_data = [float(x) for x in atom_line[start_idx:end_idx]]
                atom_data.append(selected_data)
            
            # Create DataFrame for this timestep
            df = pd.DataFrame(atom_data, columns=selected_columns)
            df['timestep'] = timestep  # Add timestep column for tracking
            data_list.append(df)
            
            i += natoms_current + 1
            continue
        
        i += 1
    
    # Combine all timesteps into single DataFrame
    # all_data = pd.concat(data_list, ignore_index=True)
    
    return {
        'timesteps': timesteps,
        'natoms': natoms_list,
        'box_bounds': box_bounds_list,
        'data': data_list,
    }

def read_all_dumps_in_folder(folder_path, columns_range=('id', 'c_pstress[6]'), 
                               file_pattern='Dump.shear_fixed_Load_1wall.*', save_combined=False,
                               output_filename=None):
    folder_path = Path(folder_path)
    dump_files = sorted(folder_path.glob(file_pattern))
    
    if len(dump_files) == 0:
        print(f"No files matching pattern '{file_pattern}' found in {folder_path}")
        return None
    
    print(f"Found {len(dump_files)} dump files in {folder_path}")
    print(f"Reading files...")
    
    all_dataframes = []
    
    for idx, dump_file in enumerate(dump_files):
        try:
            result = read_lammps_dump(str(dump_file), columns_range=columns_range)
            
            # result['data'] is a LIST of DataFrames (one per timestep)
            # If each file has only one timestep, take the first one
            if len(result['data']) > 0:
                df = result['data'][0]  # Take first (and likely only) timestep
                df = df.copy()
                df['filename'] = dump_file.name
                df['file_index'] = idx
                all_dataframes.append(df)
            
            if (idx + 1) % 100 == 0 or (idx + 1) == len(dump_files):
                print(f"  Processed {idx + 1}/{len(dump_files)} files...")
                
        except Exception as e:
            print(f"  Error reading {dump_file.name}: {e}")
            continue
    
    if len(all_dataframes) == 0:
        print("No data could be read from files")
        return None
    
    return all_dataframes  # Return list of DataFrames (one per file)

In [ ]:
#-------------------------------------------------- Calculate the Stress Tensor ------------------------------------#
# Per file computation of stress tensor #
def cauchy_stress_tensor(df, y, kernel_width):
    """
    Compute stress values from dataframe using IRVING-KIRKWOOD FORMULA
    1. Kinetic component: from vx, vy, vz
    2. Potential component: from fx, fy, fz and particle separations dx, dy, dz
    3. Use Gaussian kernel for spatial averaging
    4. Return average stress tensor over all evaluation points y
    """
    results = {
        'y': y, 
        's_xy': np.zeros(len(y)), 
        's_xx': np.zeros(len(y)), 
        's_yy': np.zeros(len(y)), 
        's_zz': np.zeros(len(y)),
        's_yz': np.zeros(len(y)), 
        's_zx': np.zeros(len(y)), 
        'pressure': np.zeros(len(y))
    }
    
    particle_volumes = (4/3) * np.pi * df['radius'].values**3
    bin_volume = (3 * kernel_width)**3
    
    for i, y_eval in enumerate(y):
        dist = np.abs(df['y'].values - y_eval)
        gaussian_weight = np.exp(-0.5 * (dist / kernel_width)**2)
        total_weight = np.sum(gaussian_weight)
        
        if total_weight < 1e-10:
            continue
            
        # Local average velocities
        vx_local = np.sum(df['vx'].values * gaussian_weight) / total_weight
        vy_local = np.sum(df['vy'].values * gaussian_weight) / total_weight
        vz_local = np.sum(df['vz'].values * gaussian_weight) / total_weight
        
        # Fluctuation velocities
        wx = df['vx'].values - vx_local
        wy = df['vy'].values - vy_local
        wz = df['vz'].values - vz_local

        avg_neighbor_distance = 2 * df['radius'].values  # approximation
        dx = avg_neighbor_distance * np.sign(df['fx'].values)
        dy = avg_neighbor_distance * np.sign(df['fy'].values)
        dz = avg_neighbor_distance * np.sign(df['fz'].values)
        
        #  =============== IRVING-KIRKWOOD FORMULA ====================
        # Kinetic component: w (*) w (tensor product of fluctuation velocities)
        # Potential component: f (*) r (tensor product of force and displacement)
        
        # All stress tensor components
        shear_xx = (wx * wx + (df['fx'].values * dx)/2) * particle_volumes / bin_volume
        shear_yy = (wy * wy + (df['fy'].values * dy)/2) * particle_volumes / bin_volume
        shear_zz = (wz * wz + (df['fz'].values * dz)/2) * particle_volumes / bin_volume
        
        shear_xy = (wx * wy + (df['fx'].values * dy)/2) * particle_volumes / bin_volume
        shear_xz = (wx * wz + (df['fx'].values * dz)/2) * particle_volumes / bin_volume
        shear_yz = (wy * wz + (df['fy'].values * dz)/2) * particle_volumes / bin_volume
        # =================================================================
        
        # Weighted average with Gaussian kernel
        sxx = np.sum(shear_xx * gaussian_weight) / total_weight
        syy = np.sum(shear_yy * gaussian_weight) / total_weight
        szz = np.sum(shear_zz * gaussian_weight) / total_weight
        
        sxy = np.sum(shear_xy * gaussian_weight) / total_weight
        sxz = np.sum(shear_xz * gaussian_weight) / total_weight
        syz = np.sum(shear_yz * gaussian_weight) / total_weight
        
        # Stress tensor is symmetric
        pressure = (sxx + syy + szz) / 3
        
        results['s_xx'][i] = sxx
        results['s_yy'][i] = syy
        results['s_zz'][i] = szz
        results['s_xy'][i] = sxy
        results['s_yz'][i] = syz
        results['s_zx'][i] = sxz
        results['pressure'][i] = pressure
    
    # Average stress tensor components over all evaluation points
    a_xx = np.mean(results['s_xx'])
    a_yy = np.mean(results['s_yy'])
    a_zz = np.mean(results['s_zz'])
    a_xy = np.mean(results['s_xy'])
    a_yz = np.mean(results['s_yz'])
    a_zx = np.mean(results['s_zx'])
    

    stress_tensor = torch.tensor([
        [a_xx, a_xy, a_zx],
        [a_xy, a_yy, a_yz],
        [a_zx, a_yz, a_zz]
    ], dtype=torch.float32)
    
    return stress_tensor

#-------------------------------------------------- Calculate the Deformation Tensor ------------------------------------#
# Per file computation of stress tensor #
def deformation_tensor(df, y, kernel_width):
    """
    Simplified deformation calculation using weighted least s quares.
    """
    shear_rates = []
    p1 = 9
    p2 = 133
    p3 = 475
    p4 = 7571.4
    Pressure = 0.0
    min_particles = 10  # Minimum particles required for calculation
    for y_eval in y:
        # Select particles near evaluation point
        dist = np.abs(df['y'].values - y_eval)
        mask = dist < 3 * kernel_width
        
        if np.sum(mask) < min_particles:
            continue
        
        y_local = df['y'].values[mask]
        vx_local = df['vx'].values[mask]
        
        # Calculate weights
        weights = np.exp(-0.5 * (dist[mask] / kernel_width) ** 2)
        total_weight = np.sum(weights)
        
        if total_weight < 1e-20:
            continue
        
        # Normalize weights
        weights_norm = weights / total_weight
        
        # Weighted means
        y_mean = np.sum(weights_norm * y_local)
        vx_mean = np.sum(weights_norm * vx_local)
        
        # Weighted covariance and variance
        y_centered = y_local - y_mean
        vx_centered = vx_local - vx_mean
        
        cov_y_vx = np.sum(weights_norm * y_centered * vx_centered)
        var_y = np.sum(weights_norm * y_centered ** 2)
        
        # Avoid division by zero
        if var_y < 1e-20:
            continue
        
        # Shear rate = d(vx)/dy ≈ cov(y, vx) / var(y)
        shear_rate = cov_y_vx / var_y

        # Sanity check
        if np.isfinite(shear_rate) and np.abs(shear_rate) < 1e6:
            shear_rates.append(shear_rate)
    deformation_tensor = torch.tensor([[0.0, np.mean(shear_rates) if len(shear_rates) > 0 else 0.0, 0.0],
                                    [np.mean(shear_rates) if len(shear_rates) > 0 else 0.0, 0.0, 0.0],
                                    [0.0, 0.0, 0.0]], dtype=torch.float32)
    return deformation_tensor

def velocity_profile_3d(df, y, kernel_width):
    """
    Compute velocity profile v(y) = [vx(y), vy(y), vz(y)] using Gaussian kernel.
    
    Returns:
        vx_profile, vy_profile, vz_profile: numpy arrays of shape (len(y),)
    """
    min_particles = 10
    n_points = len(y)
    
    vx_profile = np.zeros(n_points)
    vy_profile = np.zeros(n_points)
    vz_profile = np.zeros(n_points)
    
    for idx, y_eval in enumerate(y):
        # Select particles near evaluation point
        dist = np.abs(df['y'].values - y_eval)
        mask = dist < 3 * kernel_width
        
        if np.sum(mask) < min_particles:
            vx_profile[idx] = np.nan
            vy_profile[idx] = np.nan
            vz_profile[idx] = np.nan
            continue
        
        # Get local velocities
        vx_local = df['vx'].values[mask]
        vy_local = df['vy'].values[mask]
        vz_local = df['vz'].values[mask]
        
        # Calculate Gaussian weights
        weights = np.exp(-0.5 * (dist[mask] / kernel_width) ** 2)
        total_weight = np.sum(weights)
        
        if total_weight < 1e-20:
            vx_profile[idx] = np.nan
            vy_profile[idx] = np.nan
            vz_profile[idx] = np.nan
            continue
        
        # Normalize weights and compute weighted mean
        weights_norm = weights / total_weight
        
        vx_profile[idx] = np.sum(weights_norm * vx_local)
        vy_profile[idx] = np.sum(weights_norm * vy_local)
        vz_profile[idx] = np.sum(weights_norm * vz_local)
    
    return vx_profile, vy_profile, vz_profile

#### Main processing loop
* Reading files
* Computing Stress and Deformation 
* Saving the tensors

### Importing Libraries....

In [ ]:
import os
import torch
import glob
import numpy as np
import pandas as pd
from pathlib import Path
from torch import nn
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
import numpy as np
import pandas as pd
import torch
def Target_matrix(stress, deformation, dt):

    # Target matrix: time derivative of each components of stress: x,y,z and shear stress as well xy,yz,zx
    n_samples = stress.shape[0]
    
    target_matrix = torch.zeros(n_samples,6)

    tau_xy = torch.zeros(n_samples,1)
    
    # Compute time derivatives for all samples at once
    stress_dot = torch.zeros_like(target_matrix)
    stressXX = stress[:,0,0]
    stressYY = stress[:,1,1]
    stressZZ = stress[:,2,2]
    stressXY = stress[:,0,1]
    stressYZ = stress[:,1,2]
    stressZX = stress[:,2,0]

    Stress = torch.stack((stressXX,stressYY,stressZZ,stressXY,stressYZ,stressZX),dim=1)
    stress_dot[0] = (Stress[1] - Stress[0]) / dt
    stress_dot[-1] = (Stress[-1] - Stress[-2]) / dt
    stress_dot[1:-1] = (Stress[2:] - Stress[:-2]) / (2 * dt)
    # Convert stress_dot to full 3x3 tensor form
    stress_dot_master = torch.zeros((n_samples,3,3), dtype=torch.float32)
    stress_dot_master[:,0,0] = stress_dot[:,0]
    stress_dot_master[:,1,1] = stress_dot[:,1]
    stress_dot_master[:,2,2] = stress_dot[:,2]
    stress_dot_master[:,0,1] = stress_dot[:,3]
    stress_dot_master[:,1,0] = stress_dot[:,3]
    stress_dot_master[:,1,2] = stress_dot[:,4]
    stress_dot_master[:,2,1] = stress_dot[:,4]
    stress_dot_master[:,2,0] = stress_dot[:,5]
    stress_dot_master[:,0,2] = stress_dot[:,5]

    # Build W tensors for all samples
    W = torch.zeros_like(stress)
    shear_rates = deformation[:, 0, 1]  # Extract all shear rates
    W[:, 0, 1] = 0.5 * shear_rates
    W[:, 1, 0] = -0.5 * shear_rates
    
    # Compute Jaumann derivative for all samples
    j_t = stress_dot_master - torch.bmm(W, stress) + torch.bmm(stress, W)

    # Populating target matrix
    target_matrix[:,0] = j_t[:,0,0]
    target_matrix[:,1] = j_t[:,1,1]
    target_matrix[:,2] = j_t[:,2,2]
    target_matrix[:,3] = j_t[:,0,1] # tau_xy
    # tau_xy[:, 0] = stress[:,0,1]
    tau_xy[:,0] = j_t[:,0,1]
    target_matrix[:,4] = j_t[:,1,2]
    target_matrix[:,5] = j_t[:,2,0]
    #return target_matrix
    return tau_xy

In [ ]:
def granular_library(stress, deformation, velocity_profiles, dX):
    """
    Library of candidate terms for the stress-deformation relationship.
    Arguments:
        stress: torch.Tensor of shape [n_samples, 3, 3] - stress tensor over time
        deformation: torch.Tensor of shape [n_samples, 3, 3] - deformation tensor over time
        velocity_profiles: torch.Tensor of shape [n_samples, n_y_points, 3] - velocity at spatial points
        p: reference pressure for non-dimensionalization
        dX: spatial discretization (particle diameter or kernel width)
    Returns:
        Library: torch.Tensor of shape [n_samples, 26], 26 terms: unity, stress (all 6 independent components), deformation, laplacian of Deformation tensor, and granular temparature
        LIBRARY_TERMS: dict mapping term names to column indices
    """
    n_samples = deformation.shape[0]  # time steps
    n_y_points = velocity_profiles.shape[1]  # spatial evaluation points
    
    LIBRARY_TERMS = {
        '1': 0,
        'tau_xx': 1, 'tau_yy': 2, 'tau_zz': 3, 
        'tau_xy': 4, 'tau_yz': 5, 'tau_zx': 6,
        'D_xx': 7, 'D_yy': 8, 'D_zz': 9, 
        'D_xy': 10, 'D_yz': 11, 'D_zx': 12,
        'D_xx^2': 13, 'D_yy^2': 14, 'D_zz^2': 15, 
        'D_xy^2': 16, 'D_yz^2': 17, 'D_zx^2': 18,
        'lap_D_xx': 19, 'lap_D_yy': 20, 'lap_D_zz': 21, 
        'lap_D_xy': 22, 'lap_D_yz': 23, 'lap_D_zx': 24, 
        'T': 25, 'phi':26, 'lap_phi':27
    }
    lib_xy = {
        'tau_xy':0, 'D_xy':1, 'D_xy^2':2, 'lap_D_xy':3, 'Tg':4
    }
    
    Library = torch.zeros((n_samples, 26), dtype=torch.float32)
    theta = torch.zeros((n_samples, 5), dtype=torch.float32)
    # ==================== STRESS TERMS (Non-dimensionalized) ====================
    pressure = (stress[:, 0, 0] + stress[:, 1, 1] + stress[:, 2, 2]) / 3.0 + 1e-10
    
    tau_xx = stress[:, 0, 0] 
    tau_yy = stress[:, 1, 1]
    tau_zz = stress[:, 2, 2]
    tau_xy = stress[:, 0, 1]
    tau_yz = stress[:, 1, 2]
    tau_zx = stress[:, 2, 0]
    
    # ==================== DEFORMATION TERMS ====================
    D_xx = deformation[:, 0, 0]
    D_yy = deformation[:, 1, 1]
    D_zz = deformation[:, 2, 2]
    D_xy = deformation[:, 0, 1]
    D_yz = deformation[:, 1, 2]
    D_zx = deformation[:, 2, 0]
    
    # ==================== LAPLACIAN OF DEFORMATION (Temporal) ====================
    # Compute laplacian(D) using second-order central differences
    h2 = dX * dX
    
    def laplacian(component,h2):
        """
        Using central difference: (D[i+1] - 2*D[i] + D[i-1]) / dx*dx
        """
        lap = torch.zeros_like(component)
        lap[1:-1] = (component[2:] - 2 * component[1:-1] + component[:-2]) / h2
        # Boundary: forward/backward difference
        lap[0] = (component[1] - component[0]) / h2
        lap[-1] = (component[-1] - component[-2]) / h2
        return lap
    
    lap_D_xx = laplacian(D_xx, h2)
    lap_D_yy = laplacian(D_yy, h2)
    lap_D_zz = laplacian(D_zz, h2)
    lap_D_xy = laplacian(D_xy, h2)
    lap_D_yz = laplacian(D_yz, h2)
    lap_D_zx = laplacian(D_zx, h2)
    
    # ==================== GRANULAR TEMPERATURE ====================
    # T_g = (1/3) * <(v - <v>)^2> averaged over space and velocity components
    # Shape of velocity_profiles: [n_samples, n_y_points, 3] for (vx, vy, vz)
    
    T_g = torch.zeros(n_samples, dtype=torch.float32)
    
    for t in range(n_samples):
        # Get velocity field at time t: shape [n_y_points, 3]
        v_field = velocity_profiles[t]  # [n_y_points, 3]
        
        # Spatial average of velocity at each component
        v_mean = torch.mean(v_field, dim=0, keepdim=True)  # [1, 3]
        
        # Fluctuation velocities
        v_fluctuation = v_field - v_mean  # [n_y_points, 3]
        
        # Squared fluctuation velocity magnitude
        v_sq_fluct = torch.sum(v_fluctuation**2, dim=1)  # [n_y_points]
        
        # Granular temperature: (1/3) * spatial average of |v'|²
        T_g[t] = torch.mean(v_sq_fluct) / 3.0
    
    # Non-dimensionalize by characteristic velocity squared
    # For quasi-static flows, use: v_char² = p / ρ (if available)
    # Or normalize by maximum observed temperature
    T_g_normalized = T_g #/ (torch.max(T_g) + 1e-10)

    # ====================== local Volume fraction ==================== #
    

    
    # ==================== POPULATE LIBRARY MATRIX ==================== #
    Library[:, LIBRARY_TERMS['1']] = torch.ones(n_samples, dtype=torch.float32)
    # Stress terms
    Library[:, LIBRARY_TERMS['tau_xx']] = tau_xx
    Library[:, LIBRARY_TERMS['tau_yy']] = tau_yy
    Library[:, LIBRARY_TERMS['tau_zz']] = tau_zz
    Library[:, LIBRARY_TERMS['tau_xy']] = tau_xy
    Library[:, LIBRARY_TERMS['tau_yz']] = tau_yz
    Library[:, LIBRARY_TERMS['tau_zx']] = tau_zx
    # Linear deformation terms
    Library[:, LIBRARY_TERMS['D_xx']] = D_xx
    Library[:, LIBRARY_TERMS['D_yy']] = D_yy
    Library[:, LIBRARY_TERMS['D_zz']] = D_zz
    Library[:, LIBRARY_TERMS['D_xy']] = D_xy
    Library[:, LIBRARY_TERMS['D_yz']] = D_yz
    Library[:, LIBRARY_TERMS['D_zx']] = D_zx
    # Squared deformation terms
    Library[:, LIBRARY_TERMS['D_xx^2']] = (D_xx**2)
    Library[:, LIBRARY_TERMS['D_yy^2']] = (D_yy**2)
    Library[:, LIBRARY_TERMS['D_zz^2']] = (D_zz**2)
    Library[:, LIBRARY_TERMS['D_xy^2']] = (D_xy**2)
    Library[:, LIBRARY_TERMS['D_yz^2']] = (D_yz**2)
    Library[:, LIBRARY_TERMS['D_zx^2']] = (D_zx**2)
    # Laplacian terms
    Library[:, LIBRARY_TERMS['lap_D_xx']] = lap_D_xx
    Library[:, LIBRARY_TERMS['lap_D_yy']] = lap_D_yy
    Library[:, LIBRARY_TERMS['lap_D_zz']] = lap_D_zz
    Library[:, LIBRARY_TERMS['lap_D_xy']] = lap_D_xy
    Library[:, LIBRARY_TERMS['lap_D_yz']] = lap_D_yz
    Library[:, LIBRARY_TERMS['lap_D_zx']] = lap_D_zx
    # Granular temperature
    Library[:, LIBRARY_TERMS['T']] = T_g_normalized

    # Library only for xy component
    theta[:, lib_xy['tau_xy']] = tau_xy
    theta[:, lib_xy['D_xy']] = D_xy
    theta[:, lib_xy['D_xy^2']] = (D_xy**2)
    theta[:, lib_xy['lap_D_xy']] = lap_D_xy
    theta[:, lib_xy['Tg']] = T_g_normalized

    #return Library, LIBRARY_TERMS
    return theta, lib_xy


In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

def STRidge_optimizer(target_matrix, library_matrix, lam, iterations=1000, tol=1e-6):
    """
    STRidge optimizer for sparse regression
    
    Parameters:
    -----------
    target_matrix : torch.Tensor, shape [n_samples, 6] - flattened stress derivatives
    library_matrix : torch.Tensor, shape [n_samples, n_terms] - flattened library terms
    lam : float - regularization parameter
    
    Returns:
    --------
    coefficients : torch.Tensor, shape [n_terms, 6] - one coefficient vector per stress component
    """
    n_samples, n_terms = library_matrix.shape
    n_components = target_matrix.shape[1]  # Should be 6 or 1 for tau_xy
    
    # Initialize coefficients for each stress component
    torch.manual_seed(42)
    coefficients = torch.randn(n_terms, n_components, requires_grad=True)
    optimizer = torch.optim.Adam([coefficients], lr=0.01)
    
    for iteration in range(iterations):
        optimizer.zero_grad()
        
        # Predicted: [n_samples, n_terms] @ [n_terms, 1] -> [n_samples, 1]
        predicted = torch.mm(library_matrix, coefficients)
        
        # Ridge loss
        loss = torch.mean((predicted - target_matrix) ** 2) + lam * torch.sum(coefficients ** 2)
        loss.backward()
        optimizer.step()
        
        # Thresholding step
        with torch.no_grad():
            threshold = lam * 0.1  # Adaptive threshold
            coefficients[torch.abs(coefficients) < threshold] = 0.0
        
        # Check convergence
        if iteration > 10 and loss.item() < tol:
            break
    
    return coefficients.detach()


def optimal_matrix(target_tensor, library_tensor, lam, iterations=20, tol=1e-6):
    """
    Optimized sparse regression with train-test split
    
    Parameters:
    -----------
    target_tensor : torch.Tensor, shape [n_samples, 6] - Jaumann stress derivatives
    library_tensor : torch.Tensor, shape [n_samples, n_terms, 3, 3] - Library terms
    
    Returns:
    --------
    xhi_best : torch.Tensor, shape [n_terms, 6] - Best coefficients
    """
    
    n_samples, n_terms= library_tensor.shape
 
    # Split data
    train_idx, test_idx = train_test_split(
        np.arange(n_samples), test_size=0.2, random_state=42
    )
    
    train_target = target_tensor[train_idx]
    test_target = target_tensor[test_idx]
    train_library = library_tensor[train_idx]
    test_library = library_tensor[test_idx]
    
    # Initialize
    err_best = float('inf')
    xhi_best = None
    tol_current = tol
    d_tol = tol / 10
    
    # Initial guess using least squares
    try:
        # Solve: train_library @ xhi = train_target
        xhi_initial = torch.linalg.lstsq(train_library, train_target).solution
    except:
        xhi_initial = torch.zeros(n_terms, 6)
    
    for iter in range(iterations):
        # Get coefficients using STRidge
        w = STRidge_optimizer(train_target, train_library, lam, iterations=100, tol=tol_current)
        
        # Compute test error
        predicted_test = torch.mm(test_library, w)
        err = torch.norm(test_target - predicted_test).item()
        l0_penalty = 0.001 * torch.count_nonzero(w).item()
        total_err = err + l0_penalty
        
        # Update best
        if total_err < err_best:
            err_best = total_err
            xhi_best = w
            tol_current = tol_current + d_tol
        else:
            tol_current = max(0, tol_current - 2 * d_tol)
            d_tol = 2 * d_tol / (iterations - iter + 1)
            tol_current = tol_current + d_tol
        
        # print(f"Iteration {iter+1}/{iterations}: Error = {total_err:.6f}, L0 = {torch.count_nonzero(w).item()}")
    
    return xhi_best


In [ ]:
def print_equations(library_terms, xi):
    n_equations = xi.shape[1]  # 6 equations
    library = [''] * len(library_terms)
    for term, idx in library_terms.items():
        library[idx] = term
    for j in range(n_equations):
        terms = []
        for i in range(len(library)):
            coef = xi[i, j]
            if abs(coef) > 1e-6: 
                if coef >= 0 and terms:
                    terms.append(f"+ {coef:.6f}*{library[i]}")
                else:
                    terms.append(f"{coef:.6f}*{library[i]}")
        
        print(f"σ_{j+1} = " + " ".join(terms))

In [ ]:
from itertools import product
import random
import numpy as np
import torch
from pathlib import Path

vol_frac = ['578', '588', '601']  # 3 volume fractions
vt = ['0.0031', '0.014', '0.031', '0.085', '0.14', '0.23']  # 6 velocities
all_combinations = list(product(vol_frac, vt))
random.seed(1)
random.shuffle(all_combinations)

y_eval = np.linspace(-0.00004, 0.4, 100)
kernel_width = 0.02

# Pre-allocate 4D tensors
n_folders = len(vol_frac) * len(vt)  # 18 folders
n_y_points = len(y_eval)  # 100 spatial points

xi_coefficients = None
folder_idx = 0
xi_matrix = None
for i, j in all_combinations:
    folder_path = Path(r'F:\DATA_constant_volume_DEM\DATA_constant_volume') / f'vf{i}_vt{j}'
    
    print(f"\nProcessing folder {folder_idx+1}/{n_folders}: vf{i}_vt{j}")
    if not folder_path.exists():
        print(f"  ⚠ Folder does not exist, skipping...")
        continue
    else:
        result = read_all_dumps_in_folder(
            folder_path, 
            columns_range=('id', 'fz'), 
            file_pattern='Dump.shear_fixed_Load_1wall.*', 
            save_combined=False,
            output_filename=None
        )

        n_files = len(result)   #count the total number of files in the folder: time step dimension
        stress_data = torch.zeros((n_files, 3, 3), dtype=torch.float32)
        deformation_data = torch.zeros((n_files, 3, 3), dtype=torch.float32)
        velocity_data = torch.zeros((n_files, len(y_eval), 3), dtype=torch.float32)

        print(f"  Found {len(result)} files")

        for file_idx, dump_df in enumerate(result):
            # Get 3×3 stress tensor
            stress_3x3 = cauchy_stress_tensor(dump_df, y_eval, kernel_width)
            # Get 3×3 deformation tensor
            deform_3x3 = deformation_tensor(dump_df, y_eval, kernel_width)
            # NEW: Get velocity profile at all y evaluation points
            vx_profile, vy_profile, vz_profile = velocity_profile_3d(dump_df, y_eval, kernel_width)

            stress_data[file_idx] = stress_3x3
            deformation_data[file_idx] = deform_3x3
            velocity_data[file_idx, :, 0] = torch.from_numpy(vx_profile)
            velocity_data[file_idx, :, 1] = torch.from_numpy(vy_profile)
            velocity_data[file_idx, :, 2] = torch.from_numpy(vz_profile)
            #####
            # Since time steps are different, rather than storing, implement the regression here inside the loop for this will act as the main loop for folders
            # Nested loop will have one for stridge regression analysis to come up with optimal coefficients
            #####

            if (file_idx + 1) % 500 == 0:
                print(f"  Processed {file_idx + 1}/{len(result)} files...")

        # spacial and temporal elements
        dx = 0.02
        dt = 0.0003
        target_Matrix = Target_matrix(stress_data, deformation_data, dt)
        Library, LIBRARY_TERMS = granular_library(stress_data, deformation_data, velocity_data, dx)
        
        print(f" Target and library matrix created, with sizes (resp): ", target_Matrix.shape, Library.shape)
        print(f"  Running STRidge optimization...")
        xi_coefficients = optimal_matrix(target_Matrix, Library, 0.03, iterations=500, tol=1e-6)


        print(f" Optimal Coefficient obtained: ", xi_coefficients)
        print(f" target matrix: ", target_Matrix.sum())
        print(f"✓ Completed vf{i}_vt{j}")

        folder_idx += 1
        
print("✓ Computation completed")


In [ ]:
tol = 1e-6
sigma_t  = torch.zeros_like(target_Matrix)
x,y = target_Matrix.shape
for i in range(x):
    for j in range(y):
        if target_Matrix[i,j] >= tol:
            sigma_t[i,j] = target_Matrix[i,j]
print(sigma_t.sum(0), ' = ')
print_equations(LIBRARY_TERMS, xi_coefficients)